In [ ]:
#From https://github.com/gabrielo/Allegheny-County-Property-Assessments

In [231]:
# assessments/cd/AC Property Assessments_10012017.xls (which is actually TDF) from Amy Gottsegen and Randy Sargent buying a CD from the county assessors office
# assessments/Allegheny_County_Parcel_Boundaries.geojson from https://data.wprdc.org/dataset/allegheny-county-parcel-boundaries
import array, csv, datetime, json, math, numpy, os, random
from dateutil.parser import parse
from shapely.geometry import mapping, shape
from PIL import Image
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import pandas,numbers
from operator import itemgetter, attrgetter
import string
import calendar,time
from IPython.core.display import HTML


def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]


def GetCentroid(geometry):
    s = shape(geometry)
    return (s.centroid.x, s.centroid.y)   

def GetEpoch(date):
    return (date - datetime.datetime(1970, 1, 1)).total_seconds()

def HexToRgb(hex_string):
    rgb = colors.hex2color(hex_string)
    r,g,b = tuple([int(255*x) for x in rgb])
    return (r,g,b)

In [ ]:
# Read in assessments data as TDF
path = "assessments/cd/AC Property Assessments_10012017.xls"
apd = pandas.read_csv(path,sep='\t')

In [ ]:
# This is Gabriel's initial version using Bob Gradeck's version of the assessment data
# data = []
# with open("capture/alleghenycountymasterfile12012016.csv", "rb") as csvfile:
#     reader = csv.DictReader(csvfile)
#     for line in reader:
#         data.append(line)
# print(len(data))
# data[10]


In [ ]:
with open("assessments/Allegheny_County_Parcel_Boundaries.geojson") as f:
    parcel_json = json.load(f)
len(parcel_json['features'])

In [ ]:
print parcel_json['features'][0]['properties']['PIN']
print apd['PARID'][0]

In [ ]:
data_idx = {}
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    data_idx[par_id] = i
    
parcel_idx = {}
for i in range(0,len(parcel_json["features"])):
    pin = parcel_json['features'][i]['properties']['PIN']    
    parcel_idx[pin] = i
              

In [ ]:
centroids = {}
missed = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    if parcel_idx.has_key(par_id):
        parcel_id = parcel_idx[par_id]
        centroid = GetCentroid(parcel_json["features"][parcel_id]["geometry"])
        centroids[par_id] = centroid
    else:
        missed.append(par_id)


In [ ]:
print len(missed)
print len(centroids)

In [ ]:
points = []
missed_parcels = 0
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    if parcel_idx.has_key(par_id):
        parcel_id = parcel_idx[par_id]
        centroid = GetCentroid(parcel_json["features"][parcel_id]["geometry"])
        points += LonLatToPixelXY(centroid)
    else:
        missed_parcels += 1
array.array('f', points).tofile(open('assessments/parcels.bin', 'wb'))


In [ ]:
missed_parcels

In [ ]:
# Below is Anne's attempt to put floating point representations of colors into the .bin file for a new 
# shader Randy's writing

In [ ]:
def incrementKey(adict,key):
    if(key == '' or math.isnan(key)):
       return
    if not adict.has_key(key):
        adict[key] = 1
    else:
        adict[key] += 1

In [ ]:
def color_from_floats(r,g,b):
    return r + g * 256.0 + b * 256.0 * 256.0

def pack_color(color):
    return color['r'] + color['g'] * 256.0 + color['b'] * 256.0 * 256.0;

def parse_color(color):
    color = color.strip()
    c = color
    try:
        if c[0] == '#':
            c = c[1:]
        if len(c) == 3:
            return pack_color({'r': 17 * int(c[0:1], 16),
                               'g': 17 * int(c[1:2], 16),
                               'b': 17 * int(c[2:3], 16)})
        if len(c) == 6:
            return pack_color({'r': int(c[0:2], 16),
                               'g': int(c[2:4], 16),
                               'b': int(c[4:6], 16)})
    except:
        pass
    raise InvalidUsage('Cannot parse color <code><b>%s</b></code> from spreadsheet.<br><br>Color must be in standard web form, <code><b>#RRGGBB</b></code>, where RR, GG, and BB are each two-digit hexadecimal numbers between 00 and FF.<br><br>See <a href="https://www.w3schools.com/colors/colors_picker.asp">HTML Color Picker</a>' % color)

def parse_colors(colors):
    packed = [parse_color(color) for color in colors]
    return numpy.array(packed, dtype = numpy.float32)

In [214]:
year_built_colors = ['#ffffe0','#ffdaa3','#ffb27c','#fb8768','#eb5f5b','#d3394a','#b3152f','#8b0000']
def year_built_to_color(year):
    if (year <= 1900.0):
        return color_from_floats(64,64,64)
    elif (year < 1916.0):
        return parse_color(year_built_colors[7])       
    elif (year < 1932.0):
        return parse_color(year_built_colors[6])
    elif (year < 1948.0):
        return parse_color(year_built_colors[5])
    elif (year < 1964.0):
        return parse_color(year_built_colors[4])
    elif (year < 1980.0):
        return parse_color(year_built_colors[3])
    elif (year < 1996.0):
        return parse_color(year_built_colors[2])
    elif  (year < 2012.0):
        return parse_color(year_built_colors[1])
    elif  (year < 2016.0):
        return parse_color(year_built_colors[0])
    else:
        return color_from_floats(255,255,255)
 
   
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(year_built_to_color(float(year_built)))
        # Set start valid time as year_built, and max positive float as end valid time
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(float(1e38))
array.array('f', points).tofile(open('assessments/parcels_bltyr_color_epoch.bin', 'wb'))

In [215]:
                                     
fm_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695']
def fm_total_to_color(fm_total):
    if (fm_total < 25000.):
        return parse_color(fm_colors[0]) 
    elif (fm_total < 50000.):
        return parse_color(fm_colors[1]) 
    elif (fm_total < 75000.):
        return parse_color(fm_colors[2]) 
    elif (fm_total < 100000.):
        return parse_color(fm_colors[3]) 
    elif (fm_total < 125000.):
        return parse_color(fm_colors[4]) 
    elif (fm_total < 150000.):
        return parse_color(fm_colors[5]) 
    elif (fm_total < 200000.):
        return parse_color(fm_colors[6]) 
    elif (fm_total < 250000.):
        return parse_color(fm_colors[7]) 
    elif (fm_total < 300000.):
        return parse_color(fm_colors[8]) 
    elif (fm_total < 400000.):
        return parse_color(fm_colors[9]) 
    elif (fm_total < 500000.):
        return parse_color(fm_colors[10]) 
    elif (fm_total < 750000.):
        return parse_color(fm_colors[11]) 
    else:
        return parse_color(fm_colors[12]) 
 
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    fm_total = float(apd['FAIRMARKETTOTAL'][i])   
    
    year_built = apd['YEARBLT'][i]
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)

    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(fm_total_to_color(fm_total))
        # Set start valid time as year_built, and max positive float as end valid time
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(float(1e38))
array.array('f', points).tofile(open('assessments/parcels_fm_color_epoch.bin', 'wb'))

In [217]:
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']
class_descriptions = sorted(class_descs.keys())

def class_desc_to_color(class_description):
    index = class_descriptions.index(class_description)
    return(parse_color(parcel_colors[index]))

points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    class_description = apd['CLASSDESC'][i] 
    year_built = apd['YEARBLT'][i]

    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)

    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(class_desc_to_color(class_description))
        # Set start valid time as year_built, and max positive float as end valid time
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(float(1e38))
array.array('f', points).tofile(open('assessments/parcels_class_color_epoch.bin', 'wb'))

In [ ]:
# Here are Anne's efforts using the version with owner names to generate volume of properties owned by current owner

In [ ]:
# SALEDATE field is in '%m-%d-%Y' format (ex 10-26-2012) when present
def SaledateToEpoch(datestr):
    return calendar.timegm(time.strptime(datestr, '%m-%d-%Y'))

In [239]:
# Parse apd table into dictionaries from owner names and change addresses to parid, and from parid to other info
# about the property
owner_names_to_parids = {}
owner_changeaddrs_to_parids = {}
parid_info = {}
volume_counts = {}

# For now, only look at residential properties with known centroids in 15213
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    zipcode = apd['PROPERTYZIP'][i]
    class_description = apd['CLASSDESC'][i] 
    usedesc = apd['USEDESC'][i] 
    if (usedesc == 'VACANT LAND' or not centroids.has_key(par_id)):
        # skip this one
        continue
    
    # We want to include residential, apartments (usedesc includes APART)
    if(class_description != 'RESIDENTIAL' and (isinstance(usedesc, numbers.Number) or 'APART' not in usedesc)):
        # skip this one
        continue
        
    # This one meets our criteria
    owner_name = apd['PROPERTYOWNER'][i]
    owner_changeaddr = string.strip(str(apd['CHANGENOTICEADDRESS1'][i]))
    c2 = string.strip(str(apd['CHANGENOTICEADDRESS2'][i]))
    c3 = string.strip(str(apd['CHANGENOTICEADDRESS3'][i]))
    c4 = apd['CHANGENOTICEADDRESS4'][i]
    if(c2 != ''):
        owner_changeaddr+=", " + c2
    if(c3 != ''):
        owner_changeaddr+=", " + c3
    if(c4 != ''):
        if(isinstance(c4, numbers.Number)):
            if(not math.isnan(float(c4))):
                owner_changeaddr+=", " + str(int(c4))
        else:
            owner_changeaddr+=", " + c4
        
    if(owner_changeaddr == ''):
        # skip this one
        continue
        
    if(owner_names_to_parids.has_key(owner_name)):
        owner_names_to_parids[owner_name].append(par_id)
    else:
        owner_names_to_parids[owner_name]=[par_id]
        
    if(owner_changeaddrs_to_parids.has_key(owner_changeaddr)):
        owner_changeaddrs_to_parids[owner_changeaddr].append(par_id)
    else:
        owner_changeaddrs_to_parids[owner_changeaddr]=[par_id]

    parid_info[par_id]={'owner_name':owner_name,
                        'owner_changeaddr':owner_changeaddr,
                        'centroid':centroids[par_id],
                        'HOMESTEADFLAG':apd['HOMESTEADFLAG'][i],
                        'USEDESC':apd['USEDESC'][i],
                        'OWNERDESC':apd['OWNERDESC'][i],
                        'SALEDATE':apd['SALEDATE'][i],
                        'OWNERDESC':apd['OWNERDESC'][i],
                        'CLASSDESC':apd['CLASSDESC'][i]
                       }

In [240]:
# Add counts to parid_info based on max of owner_names_to_parids or owner_changeaddrs_to_parids
# Special case properties with HOMESTEADFLAG set to HOM to be volume=1
for par_id in parid_info.keys():
    volume=1
    if(parid_info[par_id]['HOMESTEADFLAG']!='HOM'):
        ownername_num = len(owner_names_to_parids[parid_info[par_id]['owner_name']])
        ownerchangeaddr_num = len(owner_changeaddrs_to_parids[parid_info[par_id]['owner_changeaddr']])
        volume = max(ownername_num,ownerchangeaddr_num)
    parid_info[par_id]['volume'] = volume
    incrementKey(volume_counts,volume)

In [ ]:
volume_counts

In [ ]:
for ownername in owner_names_to_parids.keys():
    ownername_num = len(owner_names_to_parids[ownername])
    if(ownername_num>10):
        print "%s: %d" % (ownername,ownername_num)

In [ ]:
for ownerchangeaddr in owner_changeaddrs_to_parids.keys():
    ownerchangeaddr_num = len(owner_changeaddrs_to_parids[ownerchangeaddr])
    if(ownerchangeaddr_num>10):
        print "%s: %d" % (ownerchangeaddr,ownerchangeaddr_num)

In [ ]:
for par_id in parid_info.keys():
    if(parid_info[par_id]['volume']>10):
        print parid_info[par_id]

In [241]:
# Write out a binary file with the volume colors
vol_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695']
def volume_to_color(volume):
    if (volume < 2):
        return parse_color(vol_colors[0]) 
    #elif (volume < 3):
    #    return parse_color(vol_colors[1]) 
    #elif (volume < 4):
    #    return parse_color(vol_colors[2]) 
    elif (volume < 5):
        return parse_color(vol_colors[3]) 
    elif (volume < 10):
        return parse_color(vol_colors[4]) 
    elif (volume < 20):
        return parse_color(vol_colors[5]) 
    elif (volume < 40):
        return parse_color(vol_colors[6]) 
    elif (volume < 60):
        return parse_color(vol_colors[7]) 
    elif (volume < 80):
        return parse_color(vol_colors[8]) 
    elif (volume < 150):
        return parse_color(vol_colors[9]) 
    elif (volume < 300):
        return parse_color(vol_colors[10]) 
    elif (volume < 500):
        return parse_color(vol_colors[11]) 
    else:
        return parse_color(vol_colors[12]) 

# Write out volume of ownership for each residential non-vacant land property
points = []
for par_id in parid_info.keys():
    centroid = parid_info[par_id]['centroid']
    color = volume_to_color(parid_info[par_id]['volume'])
    saledate = SaledateToEpoch('01-01-1900')
    saledate_raw = parid_info[par_id]['SALEDATE']
    if(not(saledate_raw == '' or (isinstance(saledate_raw, numbers.Number) and math.isnan(saledate_raw)))):
        # If not valid, leave as 1900, otherwise parse it into an epoch time
        saledate = SaledateToEpoch(saledate_raw)
        
    if(color != None):
        points += LonLatToPixelXY(centroid)        
        points.append(color)
        # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
        points.append(float(saledate))
        points.append(float(1e38))
    else:
        print "Color of " + str(parid_info[par_id]['volume']) + " is None"
array.array('f', points).tofile(open('assessments/res_volume_color_epoch.bin', 'wb'))

In [233]:
#original version
legend = '<svg width="400" height="30"><rect fill="#a50026" x="0" height="10" width="30"></rect><rect fill="#cd2827" x="30" height="10" width="30"></rect><rect fill="#e75436" x="60" height="10" width="30"></rect><rect fill="#f7804b" x="90" height="10" width="30"></rect><rect fill="#fdad61" x="120" height="10" width="30"></rect><rect fill="#fed788" x="150" height="10" width="30"></rect><rect fill="#ffffbf" x="180" height="10" width="30"></rect><rect fill="#b9e0ed" x="210" height="10" width="30"></rect><rect fill="#8dc0db" x="240" height="10" width="30"></rect><rect fill="#699fca" x="270" height="10" width="30"></rect><rect fill="#4d7db9" x="300" height="10" width="30"></rect><rect fill="#3e5aa7" x="330" height="10" width="30"></rect><text font-size="10.5px" y="29" x="0" fill="#ffffff">1</text><text font-size="10.5px" y="29" x="20"  fill="#ffffff">&nbsp;2</text><text font-size="10.5px" y="29" x="50"  fill="#ffffff">&nbsp;3</text><text font-size="10.5px" y="29" x="80"  fill="#ffffff"> 4</text><text font-size="10.5px" y="29" x="110"  fill="#ffffff"> 5</text><text font-size="10.5px" y="29" x="140"  fill="#ffffff">10</text><text font-size="10.5px" y="29" x="170"  fill="#ffffff">20</text><text font-size="10.5px" y="29" x="200"  fill="#ffffff">40</text><text font-size="10.5px" y="29" x="230"  fill="#ffffff">60</text><text font-size="10.5px" y="29" x="260"  fill="#ffffff">80</text><text font-size="10.5px" y="29" x="290"  fill="#ffffff">150</text><text font-size="10.5px" y="29" x="320"  fill="#ffffff">300</text><text font-size="10.5px" y="29" x="350"  fill="#ffffff">500</text>'
HTML(legend)

In [235]:
#modified version
legend = '<svg width="400" height="30"><rect fill="#a50026" x="0" height="10" width="30"></rect><rect fill="#f7804b" x="30"  height="10" width="30"></rect><rect fill="#fdad61" x="60"  height="10" width="30"></rect><rect fill="#fed788" x="90"  height="10" width="30"></rect><rect fill="#ffffbf" x="120" height="10" width="30"></rect><rect fill="#b9e0ed" x="150" height="10" width="30"></rect><rect fill="#8dc0db" x="180" height="10" width="30"></rect><rect fill="#699fca" x="210" height="10" width="30"></rect><rect fill="#4d7db9" x="240" height="10" width="30"></rect><rect fill="#3e5aa7" x="270" height="10" width="30"></rect><text font-size="10.5px" y="29" x="0" fill="#ffffff">1</text><text font-size="10.5px" y="29" x="20"  fill="#ffffff">&nbsp;2</text><text Font-size="10.5px" y="29" x="50"   fill="#ffffff"> 5</text><text font-size="10.5px" y="29" x="80"   fill="#ffffff">10</text><text font-size="10.5px" y="29" x="110"  fill="#ffffff">20</text><text font-size="10.5px" y="29" x="140"  fill="#ffffff">40</text><text font-size="10.5px" y="29" x="170"  fill="#ffffff">60</text><text font-size="10.5px" y="29" x="200"  fill="#ffffff">80</text><text font-size="10.5px" y="29" x="230"  fill="#ffffff">150</text><text font-size="10.5px" y="29" x="260"  fill="#ffffff">300</text><text font-size="10.5px" y="29" x="290"  fill="#ffffff">500</text>'
HTML(legend)

In [242]:
# Write out a binary file with the colors for residential owned by corporate, government, or regular owners
def ownertype_to_color(par_id):
    ownerdesc = parid_info[par_id]['OWNERDESC']
    classdesc = parid_info[par_id]['CLASSDESC']
    volume = parid_info[par_id]['volume']
    if (classdesc == "GOVERNMENT"):
        print "Government: %s, %s, %d" % (ownerdesc, classdesc,volume)
        # Same color as government in property class view
        return parse_color("#d1947a") 
    elif ('CORPORATION' in ownerdesc):
        # Same color as 500+ properties in volume view
        #return parse_color('#313695') 
        return parse_color('#ffffbf')
    elif ('REGULAR' in ownerdesc):
        # Regular owner, what volume?
        if(volume == 1):
            # Same color as 1 in volume view
            return parse_color('#a50026')
        else:
            # Same color as other in class view
            return parse_color('#02ca75')
    else:
        print "Unrecognized owner type: %s, %s, %d" % (ownerdesc, classdesc,volume)
        return parse_color('#303030') 

# Write out volume of ownership for each residential non-vacant land property
points = []
for par_id in parid_info.keys():
    centroid = parid_info[par_id]['centroid']
    color = ownertype_to_color(par_id)
    saledate = SaledateToEpoch('01-01-1900')
    saledate_raw = parid_info[par_id]['SALEDATE']
    if(not(saledate_raw == '' or (isinstance(saledate_raw, numbers.Number) and math.isnan(saledate_raw)))):
        # If not valid, leave as 1900, otherwise parse it into an epoch time
        saledate = SaledateToEpoch(saledate_raw)
        
    if(color != None):
        points += LonLatToPixelXY(centroid)        
        points.append(color)
        # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
        points.append(float(saledate))
        points.append(float(1e38))
    else:
        print "Color of " + str(parid_info[par_id]['volume']) + " is None"
array.array('f', points).tofile(open('assessments/ownertype_color_epoch.bin', 'wb'))

In [227]:
parid_info['0002K00028000000']

KeyError: '0002K00028000000'

In [ ]:
# Below here are modified versions of the blocks from Gabriel's original python notebook,
# none of which are needed anymore

In [ ]:
sale_prices = {}
for i in range(0, len(apd.index)):
    sale_price = apd['SALEPRICE'][i]
    if not sale_prices.has_key(sale_price):
        sale_prices[sale_price] = 1
    else:
        sale_prices[sale_price] += 1


In [ ]:
len(sale_prices.keys())


In [ ]:
years_built = {}
sale_dates = {}
fairmarket_totals = {}
classes = {}
class_descs = {}

def incrementKey(adict,key):
    if(key == '' or math.isnan(key)):
       return
    if not adict.has_key(key):
        adict[key] = 1
    else:
        adict[key] += 1
    
for i in range(0, len(apd.index)):
    year_built = apd['YEARBLT'][i]
    incrementKey(years_built, year_built)
    sale_date = apd['SALEDATE'][i]
    incrementKey(sale_dates, sale_date)
    fairmarket_total = apd['FAIRMARKETTOTAL'][i]
    incrementKey(fairmarket_totals, fairmarket_total)
    class_ = apd['CLASS'][i]
    incrementKey(classes, class_)
    class_desc = apd['CLASSDESC'][i]
    incrementKey(class_descs, class_desc)


In [ ]:
i

In [ ]:
len(apd.index)

In [ ]:
len(years_built.keys())
years_built['']

In [ ]:
years_built

In [ ]:
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
array.array('f', points).tofile(open('assessments/parcels_yr.bin', 'wb'))


In [ ]:
year_built
date = datetime.datetime.strptime('2000', '%Y')   
GetEpoch(date)

In [ ]:
fairmarket_totals['0']

In [ ]:
classes

In [ ]:
class_descs

In [ ]:
sorted(class_descs.keys())

In [ ]:
no_year_built = 0
no_= 0
for i in range(0, len(apd.index)):
    year_built = apd['YEARBLT'][i]
    class_ = apd['CLASS'][i]
    if year_built == '' and class_ == 'R':
        no_year_built += 1
    elif year_built == '':
        no_ += 1
print no_year_built
print no_

In [ ]:
class_descriptions = sorted(class_descs.keys())
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    class_description = apd['CLASSDESC'][i]  
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(class_descriptions.index(class_description))
array.array('f', points).tofile(open('assessments/parcels_yrblt.bin', 'wb'))


In [ ]:
foo = sorted(class_descs.keys())

In [ ]:
len(foo)

In [ ]:
parcel_colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628']

In [ ]:
for color in hex_colors:
    rgb_color = HexToRgb(color)
    print "%s,%s,%s" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0) 

In [ ]:
hex_colors = ["#fb3059",
"#fe6b2d",
"#d1947a",
"#c6a900",
"#02ca75",
"#00a2de",
"#9529b1"]
for color in hex_colors:
    rgb_color = HexToRgb(color)
    print "%.4f,%.4f,%.4f" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0) 

In [ ]:
hex_colors = ['#ffffff','#fffffa','#fffdf5','#fffdf1','#fffcef','#fffbed','#fffbea','#fff9e6','#fff9e3','#fff7e0','#fff7dd','#fff6db','#fff5d8','#fff4d6','#fff3d4','#fff3d2','#fff2d0','#fff0cd','#fff0cb','#ffefc8','#ffeec6','#ffedc3','#ffecc1','#ffeabe','#ffeabc','#ffe9b9','#ffe7b7','#ffe7b4','#ffe6b2','#ffe4b0','#ffe4ad','#ffe3ab','#ffe1a8','#ffe1a6','#ffe0a4','#ffdea1','#ffde9f','#ffdc9d','#ffdc9b','#ffda98','#ffd996','#ffd994','#ffd891','#ffd68f','#ffd68d','#ffd58a','#ffd388','#ffd386','#ffd284','#ffd081','#ffcf7f','#ffcf7d','#ffcd7b','#ffcc78','#ffcc76','#ffca74','#ffc971','#ffc96f','#ffc76d','#ffc76b','#ffc568','#ffc566','#ffc464','#ffc361','#ffc15d','#ffc05a','#ffbf58','#ffbe56','#ffbc53','#ffbc51','#ffba4e','#ffba4c','#ffb849','#ffb747','#ffb644','#ffb53f','#ffb43d','#ffb23a','#ffb237','#ffb134','#ffaf31','#ffaf2e','#ffad2a','#ffab24','#ffab21','#ffa91c','#ffa818','#ffa813','#ffa60b','#ffa400','#ffa300','#ffa200','#ffa100','#ff9f00','#ff9e00','#ff9d00','#ff9c00','#ff9b00','#ff9900','#ff9800','#ff9700','#ff9500','#ff9500','#ff9300','#ff9100','#ff9000','#ff8f00','#ff8d00','#ff8c00','#ff8a00','#ff8a00','#fe8800','#fe8800','#fe8700','#fe8400','#fd8400','#fd8300','#fd8200','#fd8000','#fc7f00','#fc7d00','#fc7c00','#fc7b00','#fb7a00','#fb7900','#fb7800','#fa7700','#fa7400','#fa7400','#f97200','#f97100','#f87000','#f86e00','#f86e00','#f76d00','#f76c00','#f66a00','#f66900','#f56800','#f56600','#f46600','#f46400','#f36400','#f36200','#f36100','#f26000','#f15e00','#f15d00','#f05d00','#ef5b00','#ef5900','#ef5900','#ee5800','#ed5700','#ed5500','#ec5500','#eb5400','#ea5200','#ea5100','#e95000','#e84f00','#e84e00','#e74c00','#e74b00','#e64a00','#e54900','#e44800','#e44700','#e34600','#e34500','#e14400','#e14300','#e04100','#df4000','#de4000','#de3e00','#dd3d00','#dc3d00','#db3c00','#db3a00','#da3900','#d83800','#d83700','#d73600','#d63500','#d63401','#d43301','#d43201','#d23101','#d13001','#d12e01','#d02e01','#d02d01','#ce2c01','#ce2b01','#cd2a01','#cc2901','#ca2801','#c92701','#c92601','#c82501','#c72401','#c62301','#c52201','#c42102','#c32002','#c21f02','#c01e02','#bf1d02','#bf1c02','#be1b02','#bd1a02','#bc1902','#ba1902','#ba1802','#b81702','#b81602','#b71502','#b51402','#b51302','#b41202','#b21202','#b11102','#b01002','#af0e02','#ae0d02','#ac0d02','#ac0c02','#aa0b02','#a90a02','#a80902','#a80902','#a60802','#a50702','#a30602','#a30602','#a20502','#a00502','#a00402','#9e0302','#9c0302','#9b0302','#9b0201','#9a0201','#990201','#970101','#950101','#950101','#940101','#920001','#920001','#900001','#8f0000','#8d0000','#8d0000','#8b0000']

img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(img.size[0]):    # for every pixel:
    for j in range(img.size[1]):
        pixels[i,j] = hex_to_rgb(hex_colors[i]) # set the colour accordingly
img.show
img.save("assessments/year-built-color-map.png")


In [ ]:
hex_stops =  ['#1d1e4e','#482045','#6f1f4b','#8d2747','#aa3f4c','#a9513f','#ba663a','#cf8139','#e59f44','#e9bb76']
hex_colors = ['#1d1e4e','#201e4e','#241e4d','#261e4d','#291e4c','#291f4c','#2b1f4c','#2d1f4b','#301f4b','#331f4a','#341f4a','#371f49','#381f49','#3b1f48','#3d1f48','#3d1f48','#3f2047','#402047','#432046','#452046','#462046','#482045','#492045','#4b2045','#4d2046','#4d2046','#502046','#502046','#522047','#532047','#552047','#562047','#572047','#5a2048','#5b2048','#5c2048','#5d2048','#5e2048','#602049','#622049','#632049','#652049','#65204a','#67204a','#69204a','#6b1f4a','#6b1f4a','#6c1f4b','#6f1f4b','#701f4b','#711f4b','#72204b','#72204b','#75204a','#75214a','#76214a','#77214a','#78214a','#7a224a','#7a224a','#7d2249','#7d2249','#7e2349','#7f2349','#802449','#812449','#832448','#842548','#852548','#862548','#882548','#882648','#892648','#8b2647','#8b2747','#8d2747','#8e2847','#8e2847','#902947','#902948','#912a48','#922b48','#932c48','#932c48','#952d48','#952d48','#952e49','#972f49','#973049','#983049','#993149','#9a3249','#9a3249','#9c334a','#9c334a','#9d354a','#9d354a','#9f364a','#9f364a','#a0374a','#a1384a','#a2384b','#a2394b','#a33a4b','#a43a4b','#a43b4b','#a63c4b','#a73c4b','#a83d4c','#a83d4c','#a93e4c','#a93f4c','#aa414b','#aa424a','#aa4349','#aa4448','#aa4747','#aa4846','#a94b44','#a94b43','#a94d43','#a94e41','#a94f40','#a9513f','#a9523f','#aa523f','#ab533e','#ac543e','#ac543e','#ad563e','#ad563e','#ae573e','#ae583e','#af593d','#b05a3d','#b05a3d','#b15b3d','#b15b3d','#b25d3c','#b35d3c','#b35e3c','#b45f3c','#b55f3c','#b5603c','#b6623b','#b6623b','#b7623b','#b8643b','#b9643b','#b9653a','#ba653a','#ba673a','#bb683a','#bb683a','#bc683a','#bd693a','#bd6a3a','#bd6a3a','#be6c3a','#bf6c3a','#c06d3a','#c16e3a','#c16e3a','#c16f3a','#c2713a','#c2713a','#c3723a','#c4733a','#c5743a','#c5753a','#c5753a','#c6753a','#c7773a','#c8773a','#c87939','#c97939','#c97939','#ca7a39','#ca7b39','#cb7c39','#cc7d39','#cc7e39','#cd7e39','#ce7f39','#ce8039','#cf8139','#cf8139','#d08239','#d0833a','#d1843a','#d2843a','#d2853b','#d2863b','#d3873b','#d4883b','#d5883c','#d5893c','#d58a3c','#d68b3c','#d78c3d','#d78d3d','#d88d3d','#d88d3e','#d98f3e','#da8f3e','#da903f','#da913f','#db923f','#dc933f','#dc9340','#dd9340','#dd9440','#de9641','#df9641','#df9741','#e09741','#e09942','#e19942','#e19a42','#e29a42','#e29c43','#e39d43','#e49d43','#e59e44','#e59e44','#e59f45','#e5a047','#e6a149','#e6a24b','#e6a44c','#e6a54e','#e6a650','#e6a652','#e6a652','#e7a754','#e7a855','#e7aa57','#e7ab59','#e7ab5b','#e7ac5d','#e8ae5e','#e8af60','#e8af62','#e8b063','#e8b165','#e8b265','#e8b367','#e8b469','#e8b56b','#e9b66c','#e9b76e','#e9b870','#e9b872','#e9ba73','#e9ba75','#e9bb76']

In [ ]:
hex_colors.index(hex_stops[9])

In [ ]:
years_built

In [ ]:
hex_stops = ['#f7fbff','#deebf7','#c6dbef','#9ecae1','#6baed6','#4292c6','#2171b5','#08519c','#08306b']

In [ ]:
for i in hex_stops:
  print i + ","

In [ ]:
hex_stops = ['#ffffe0','#dcfac5','#b8f4ab','#90ee90','#6dc88c','#46a386','#008080','#265a81','#233381','#000080']
for color in hex_stops:
    rgb_color = HexToRgb(color)
    print "%.4f,%.4f,%.4f" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0) 

In [ ]:
hex_stops = ['#1d1e4e','#5f2049','#8c2747','#aa414b','#ba653a','#d0823a','#e5a045','#ebc07f','#f7e0af','#ffffe0']
years = [1880,1895,1910,1925,1940,1955,1970,1985,2000,2015]

In [ ]:
for i in range(0,10):
    print "if (year < %s) {" % (years[i])
    rgb_color = HexToRgb(hex_stops[i])
    print "  color = vec4(%.4f,%.4f,%.4f,1.0)" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0)
    print "}"

In [ ]:
len(years)

In [ ]:
hex_colors = ['#ffffe0','#ffeec1','#ffdea7','#ffcb91','#ffb880','#ffa474','#fe906a','#f87d64','#f06a5e','#e75758','#db4551','#cf3447','#c0223b','#b0122c','#9e051b','#8b0000']
img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(img.size[0]):    # for every pixel:
    for j in range(img.size[1]):
        pixels[i,j] = hex_to_rgb(hex_colors[i/16]) # set the colour accordingly
img.show()
img.save("assessments/year-built-color-map.png")



In [ ]:
parcel_colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628']
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']

year_built_colors = ['#ffffe0','#ffeec1','#ffdea7','#ffcb91','#ffb880','#ffa474','#fe906a','#f87d64','#f06a5e','#e75758','#db4551','#cf3447','#c0223b','#b0122c','#9e051b','#8b0000']
year_built_colors = ['#ffffe0','#ffdaa3','#ffb27c','#fb8768','#eb5f5b','#d3394a','#b3152f','#8b0000']
img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(0,len(parcel_colors)):
    pixels[0,i] = HexToRgb(parcel_colors[i]) # set the colour accordingly
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    pixels[1,j] = HexToRgb(year_built_colors[i]) # set the colour accordingly
    j += 1
#img.show()
img.save("assessments/color-map.png")


In [ ]:
class_descriptions = sorted(class_descs.keys())
points = []
array.array('f', points).tofile(open('parcels_class.bin', 'wb'))
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    class_description = apd['CLASSDESC'][i]   
    if year_built == '' or year_built == '0':
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(float(year_built))
        points.append(class_descriptions.index(class_description)+1.0)
array.array('f', points).tofile(open('assessments/parcels_class.bin', 'wb'))


In [ ]:
sorted(class_descs.keys())

In [ ]:
class_descriptions.index('RESIDENTIAL')

In [ ]:
points[10000:10005]

In [ ]:
def scale_year(year):
    if year < 1800.0: 
        year = 1800.0
    return int(((year - 1800.0) * 8.0) / (2016.0-1800.))


In [ ]:
scale_year(1800.0)

In [ ]:
len(year_built_colors)

In [ ]:
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    print '<rect fill="%s" x="%s" height="10" width="25" stroke-width="1px" stroke="#666"></rect>' % (year_built_colors[i],j*25)
    j += 1
    
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    print '<text font-size="10.5px" fill="%s" y="29" x="%s">%s</text>' % (year_built_colors[i], j*25, 1800)
    j += 1


In [ ]:
list(reversed(range(10)))

In [ ]:
len(fairmarket_totals)

In [ ]:
fm_keys = sorted(map(int,fairmarket_totals.keys()))

In [ ]:
numpy.average(fm_keys)

In [ ]:
numpy.mean(fm_keys)

In [ ]:
numpy.std(fm_keys)

In [ ]:
fairmarket_totals['50000']

In [ ]:
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']
year_built_colors = ['#ffffe0','#ffdaa3','#ffb27c','#fb8768','#eb5f5b','#d3394a','#b3152f','#8b0000']
fm_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695'
]

img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(0,len(parcel_colors)):
    pixels[0,i] = HexToRgb(parcel_colors[i]) # set the colour accordingly
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    pixels[1,j] = HexToRgb(year_built_colors[i]) # set the colour accordingly
    j += 1
for i in range(0,len(fm_colors)):
    pixels[2,i] = HexToRgb(fm_colors[i]) # set the colour accordingly
    
#img.show()
img.save("assessments/color-map.png")


In [ ]:
class_descriptions = sorted(class_descs.keys())
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    class_description = apd['CLASSDESC'][i] 
    fm_total = float(apd['FAIRMARKETTOTAL'][i])    
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(float(year_built))
        points.append(class_descriptions.index(class_description)+1.0)
        points.append(fm_total)
array.array('f', points).tofile(open('assessments/parcels_all.bin', 'wb'))


In [ ]:
sorted(class_descs.keys())

In [ ]:
# Below is Anne's attempt to put floating point representations of colors into the .bin file for a new 
# shader Randy's writing

In [ ]:
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']
class_descriptions = sorted(class_descs.keys())

def class_desc_to_color(class_description):
    index = class_descriptions.index(class_description)
    return(parse_color(parcel_colors[index]))

points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    class_description = apd['CLASSDESC'][i] 
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(class_desc_to_color(class_description))
array.array('f', points).tofile(open('assessments/parcels_class_color.bin', 'wb'))

In [ ]:
len(fm_colors)

In [ ]:
fairmarket_totals['0']

In [ ]:
plt.plot(map(int,fairmarket_totals.keys()), fairmarket_totals.values(), 'ro')
plt.show()

In [ ]:
fm_keys[len(fm_keys) - 100:len(fm_keys)]

In [ ]:
fm_values = ["0", "25K", "50K", "75K", "100K", "125K", "150K", "200K", "250K", "300K", "400K", "500K", "+500K"]
for i in range(len(fm_colors)):
    print '<rect fill="%s" x="%s" height="10" width="30"></rect>' % (fm_colors[i],i*30)
    
for i in range(len(fm_values)):
    print '<text font-size="10.5px" y="29" x="%s">%s</text>' % (i*30, fm_values[i])
